In [11]:
!pip install Keras-Preprocessing

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Dense
from tensorflow.keras.models import Model
import numpy as np 
import pandas as pd
import os
import re

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from keras.preprocessing.text import Tokenizer

from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Conv1D, MaxPooling1D, Bidirectional, Concatenate, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [13]:

from sklearn.model_selection import train_test_split

from keras.layers import Input, Embedding, LSTM, Dropout, Dense
from keras.callbacks import EarlyStopping

In [14]:
df = pd.read_csv('/kaggle/input/answerscript4/p8.csv')

In [15]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
df_ind = df.loc[df['essay_set']==8]

df_ind = df_ind[['essay','domain1_score']]

df_ind = df_ind.sort_values(by= ["domain1_score"], ascending=False)
answer_sheet = df_ind.iloc[0]['essay']
df_ind = df_ind.drop(0)
students_answers = list(df['essay'].values)
marks_org = list(df['domain1_score'].values)
t=max(marks_org)
#df.apply(preprocess_text1,pandas column name)


In [17]:
df.head()

,Unnamed: 0,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,12255,20716,8,A long time ago when I was in third grade I h...,18.0,16.0,NaN,34.0,NaN,NaN,...,4.0,4.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,12256,20717,8,Softball has to be one of the single most gre...,21.0,26.0,46.0,46.0,NaN,NaN,...,6.0,6.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0
2,12257,20718,8,"Some people like making people laugh, I love ...",15.0,20.0,40.0,40.0,NaN,NaN,...,5.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
3,12258,20719,8,"""LAUGHTER"" @CAPS1 I hang out with my friends...",12.0,20.0,30.0,30.0,NaN,NaN,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0
4,12259,20721,8,Well ima tell a story about the time i got @CA...,11.0,15.0,NaN,26.0,NaN,NaN,...,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:


# Set the random seed for reproducibility
np.random.seed(42)

# Set the maximum sequence length and embedding dimension
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100

# Set the number of LSTM units and dropout rate
NUM_LSTM_UNITS = 128
DROPOUT_RATE = 0.2

# Set the batch size and number of epochs
BATCH_SIZE = 64
EPOCHS = 10

# Define the function to preprocess the text
def preprocess_text(x, remove_stopwords=False):
    x = x.lower()
    x = re.sub("[^a-z\s+]","",x)
    if remove_stopwords:
        x = " ".join([word for word in x.split() if word not in stopwords.words('english')])
    return x

# Load the essays dataset
essays_df = pd.read_csv('/kaggle/input/answerscript4/p7.csv', encoding='latin-1')

# Remove essays that have a domain1_score of NaN
essays_df = essays_df[~essays_df['domain1_score'].isna()]

# Remove stopwords from the essays
essays_df['essay'] = essays_df['essay'].apply(preprocess_text, remove_stopwords=True)
essays_df['expected']=answer_sheet


In [19]:
# Split the dataset into training, validation, and test sets
train_df, test_df = train_test_split(essays_df, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

# Tokenize the texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['essay'])

# Convert the texts to sequences and pad them to the specified maximum length
X_expected_train = pad_sequences(tokenizer.texts_to_sequences(train_df['expected']), maxlen=MAX_SEQUENCE_LENGTH)
X_actual_train = pad_sequences(tokenizer.texts_to_sequences(train_df['essay']), maxlen=MAX_SEQUENCE_LENGTH)
y_train = train_df['domain1_score'].values /t

X_expected_val = pad_sequences(tokenizer.texts_to_sequences(val_df['expected']), maxlen=MAX_SEQUENCE_LENGTH)
X_actual_val = pad_sequences(tokenizer.texts_to_sequences(val_df['essay']), maxlen=MAX_SEQUENCE_LENGTH)
y_val = val_df['domain1_score'].values /t

X_expected_test = pad_sequences(tokenizer.texts_to_sequences(test_df['expected']), maxlen=MAX_SEQUENCE_LENGTH)
X_actual_test = pad_sequences(tokenizer.texts_to_sequences(test_df['essay']), maxlen=MAX_SEQUENCE_LENGTH)
y_test = test_df['domain1_score'].values /t

# Define the inputs and embedding layer
expected_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='expected_input')
actual_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='actual_input')
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)

# Encode the inputs with the embedding layer
expected_encoded = embedding_layer(expected_input)
actual_encoded = embedding_layer(actual_input)



In [20]:
# Define the LSTM layer
lstm_layer = LSTM(NUM_LSTM_UNITS)

# Define the dropout layer
dropout_layer = Dropout(DROPOUT_RATE)

# Define the output layer
output_layer = Dense(1, activation='sigmoid')

# Encode the expected and actual inputs
expected_encoded = embedding_layer(expected_input)
actual_encoded = embedding_layer(actual_input)

# Pass the expected and actual inputs through the LSTM layer
expected_output = lstm_layer(expected_encoded)
actual_output = lstm_layer(actual_encoded)

# Apply dropout to the LSTM outputs
expected_output = dropout_layer(expected_output)
actual_output = dropout_layer(actual_output)

# Pass the LSTM outputs through the output layer
expected_output = output_layer(expected_output)
actual_output = output_layer(actual_output)


model_inputs = [expected_input, actual_input]
model_outputs = [actual_output]

# Create the model
model = Model(inputs=model_inputs, outputs=model_outputs)

# Compile the model with MAE as the loss function
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])

# Train the model
model.fit(x=[X_expected_train, X_actual_train], y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=([X_expected_val, X_actual_val], y_val))

# Evaluate the model
loss, mae = model.evaluate([X_expected_test, X_actual_test], y_test, batch_size=BATCH_SIZE)
print('Test Loss:', loss)
print('Test MAE:', mae)

Epoch 1/10
20/20 [==============================] - 33s 2s/step - loss: 0.1218 - mae: 0.1218 - val_loss: 0.0631 - val_mae: 0.0631
Epoch 2/10
20/20 [==============================] - 30s 2s/step - loss: 0.0598 - mae: 0.0598 - val_loss: 0.0590 - val_mae: 0.0590
Epoch 3/10
20/20 [==============================] - 31s 2s/step - loss: 0.0457 - mae: 0.0457 - val_loss: 0.0457 - val_mae: 0.0457
Epoch 4/10
20/20 [==============================] - 33s 2s/step - loss: 0.0349 - mae: 0.0349 - val_loss: 0.0473 - val_mae: 0.0473
Epoch 5/10
20/20 [==============================] - 33s 2s/step - loss: 0.0274 - mae: 0.0274 - val_loss: 0.0485 - val_mae: 0.0485
Epoch 6/10
20/20 [==============================] - 32s 2s/step - loss: 0.0242 - mae: 0.0242 - val_loss: 0.0475 - val_mae: 0.0475
Epoch 7/10
20/20 [==============================] - 31s 2s/step - loss: 0.0220 - mae: 0.0220 - val_loss: 0.0487 - val_mae: 0.0487
Epoch 8/10
20/20 [==============================] - 30s 2s/step - loss: 0.0201 - mae: 0.02